In [1]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer
from sklearn.utils import shuffle

from tools import plot_confusion_matrix

In [2]:
df = pd.read_csv('../data/datatran2017.csv', sep=';', encoding="latin1")
df.shape

(89518, 30)

In [3]:
df.classificacao_acidente.value_counts()

Com Vítimas Feridas    53647
Sem Vítimas            30689
Com Vítimas Fatais      5182
Name: classificacao_acidente, dtype: int64

In [4]:
df1 = pd.read_csv('../data/datatran2016_atual.csv', sep=';', encoding="latin1")
df1.shape

(96363, 25)

In [5]:
def fix_date(date):
    l = date.split('/')
    return '20{}-{}-{}'.format(l[2], l[1], l[0])
    

data = df1.data_inversa.apply(lambda x: fix_date(x))
df1.data_inversa = data

In [6]:
df1.classificacao_acidente.value_counts()

Com Vítimas Feridas    54873
Sem Vítimas            34478
Com Vítimas Fatais      5355
Ignorado                1656
Name: classificacao_acidente, dtype: int64

In [7]:
df_completo = pd.concat([df, df1])

In [8]:
df_completo.tail()

,br,causa_acidente,classificacao_acidente,condicao_metereologica,data_inversa,delegacia,dia_semana,fase_dia,feridos,feridos_graves,...,pessoas,regional,sentido_via,tipo_acidente,tipo_pista,tracado_via,uf,uop,uso_solo,veiculos
96358,163.0,Falta de atenção,Com Vítimas Fatais,Ceu Claro,2016-11-10,NaN,Quinta,Pleno dia,1,0,...,2,NaN,Crescente,Saída de Pista,Simples,Reta,MT,NaN,Urbano,1
96359,428.0,Ingestão de álcool,Com Vítimas Feridas,Ceu Claro,2016-01-01,NaN,Sexta,Plena noite,1,1,...,3,NaN,Decrescente,Colisão traseira,Simples,Reta,PE,NaN,Urbano,2
96360,267.0,Velocidade incompatível,Com Vítimas Feridas,Chuva,2016-12-10,NaN,Sábado,Pleno dia,1,0,...,1,NaN,Decrescente,Saída de Pista,Simples,Curva,MG,NaN,Rural,1
96361,393.0,Defeito mecânico em veículo,Sem Vítimas,Ceu Claro,2016-12-28,NaN,Quarta,Pleno dia,0,0,...,3,NaN,Crescente,Colisão lateral,Simples,Reta,RJ,NaN,Urbano,2
96362,40.0,Outras,Com Vítimas Feridas,Chuva,2016-12-04,NaN,Domingo,Anoitecer,5,2,...,13,NaN,Crescente,Capotamento,Dupla,Curva,RJ,NaN,Rural,1


In [9]:
df_completo.classificacao_acidente.value_counts()

Com Vítimas Feridas    108520
Sem Vítimas             65167
Com Vítimas Fatais      10537
Ignorado                 1656
Name: classificacao_acidente, dtype: int64

In [10]:
df = df_completo[df_completo.classificacao_acidente!='Ignorado']
df.shape

(184225, 30)

In [11]:
df['hour'] = df.horario.apply(lambda x: x.split(':')[0])
df['year'] = df.data_inversa.apply(lambda x: x.split('-')[0])
df['month'] = df.data_inversa.apply(lambda x: x.split('-')[1])
df['day'] = df.data_inversa.apply(lambda x: x.split('-')[2])

df.drop(['horario', 'data_inversa', 'id'], axis=1, inplace=True)

/Users/leportella/.virtualenvs/udacity/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/leportella/.virtualenvs/udacity/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/leportella/.virtualenvs/udacity/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [12]:
value = {
    'Com Vítimas Feridas': 1,
    'Sem Vítimas': 0,
    'Com Vítimas Fatais': 2
}

In [13]:
df = df[~df.classificacao_acidente.isnull()]
df.shape

(184224, 31)

In [14]:
df['target'] = df.classificacao_acidente.apply(lambda x: value[x])
df.shape

(184224, 32)

In [15]:
weekdays = {
    'sábado': 'saturday',
    'domingo': 'sunday',
    'sexta-feira': 'friday',
    'sexta': 'friday',
    'segunda-feira': 'monday',
    'segunda': 'monday',
    'quinta-feira': 'thursday',
    'quinta': 'thursday',
    'quarta-feira': 'wednesday',
    'quarta': 'wednesday',
    'terça-feira': 'tuesday',
    'terça': 'tuesday',
}

df['weekday'] = df.dia_semana.apply(lambda x: weekdays[x.lower()])

In [16]:
accident_type = {
    'saída de leito carroçável': 'carriage bed outlet',
    'saída de pista': 'out of road',
    'colisão traseira': 'rear collision',             
    'colisão lateral': 'lateral collision',
    'colisão transversal': 'transversal collision',
    'colisão com objeto estático': 'collision with static object',
    'colisão com objeto fixo': 'collision with static object',
    'colisão frontal': 'frontal collision',
    'colisão com objeto em movimento': 'collision with moving object',
    'colisão com objeto móvel': 'collision with moving object',
    'colisão com bicicleta': 'bike collision',

    'tombamento': 'tipping',

    'atropelamento de animal': 'animal trampling', 
    'atropelamento de pedestre': 'pedestrian trampling',
    'atropelamento de pessoa': 'pedestrian trampling',

    'queda de ocupante de veículo': 'vehicle occupant fall',
    'queda de motocicleta / bicicleta / veículo': 'vehicle fall',

    'engavetamento': 'pestle',      
    'incêndio': 'fire',
    'derramamento de carga': 'cargo spill',
    'danos eventuais': 'eventual damage',
    'capotamento': 'rollover',
}

df['accident_type'] = df.tipo_acidente.apply(lambda x: accident_type[x.lower().strip()])

In [17]:
accident_type_simple = {
    'saída de leito carroçável': 'carriage bed outlet',
    'saída de pista': 'out of road',
    'colisão traseira': 'collision',             
    'colisão lateral': 'collision',
    'colisão transversal': 'collision',
    'colisão com objeto estático': 'collision',
    'colisão com objeto fixo': 'collision',
    'colisão frontal': 'collision',
    'colisão com objeto em movimento': 'collision',
    'colisão com objeto móvel': 'collision',
    'colisão com bicicleta': 'collision',

    'tombamento': 'tipping',

    'atropelamento de animal': 'trampling', 
    'atropelamento de pedestre': 'trampling',
    'atropelamento de pessoa': 'trampling',

    'queda de ocupante de veículo': 'vehicle occupant fall',
    'queda de motocicleta / bicicleta / veículo': 'vehicle fall',

    'engavetamento': 'pestle',      
    'incêndio': 'fire',
    'derramamento de carga': 'cargo spill',
    'danos eventuais': 'eventual damage',
    'capotamento': 'rollover',
}

df['accident_type_simplified'] = df.tipo_acidente.apply(lambda x: accident_type_simple[x.lower().strip()])

In [18]:
accident_cause = {
 'fenômenos da natureza': 'nature phenomena',
 'falta de atenção à condução': 'lack of attention to driving',
 'animais na pista': 'animal in the road',
 'avarias e/ou desgaste excessivo no pneu': 'problems on the tire',
 'ingestão de álcool': 'alcohol ingestion',
 'defeito mecânico no veículo': 'mecanical problem',
 'desobediência às normas de trânsito pelo condutor': 'disobedience to the rules of transit by the driver',
 'velocidade incompatível': 'incompatible speed',
 'restrição de visibilidade': 'visibility restriction',
 'falta de atenção do pedestre': 'lack of attention of the pedestrian',
 'condutor dormindo': 'driver asleep',
 'pista escorregadia': 'slippery track',
 'não guardar distância de segurança': 'unsafety distance',
 'ultrapassagem indevida': 'improper overshoot',
 'defeito na via': 'defect on the road',
 'sinalização da via insuficiente ou inadequada': 'insufficient or inadequate road signs',
 'mal súbito': 'sudden onset',
 'carga excessiva e/ou mal acondicionada': 'excessive cargo',
 'objeto estático sobre o leito carroçável': 'static object on the dockable bed',
 'deficiência ou não acionamento do sistema de iluminação/sinalização do veículo': 'no lights on the vehicle',
 'ingestão de substâncias psicoativas': 'drugs ingestion',
 'agressão externa': 'external offensive',
 'falta de atenção': 'lack of attention',
 'ingestão de álcool': 'alcohol ingestion',
 'outras': 'other',
 'velocidade incompatível': 'incompatible speed',
 'ultrapassagem indevida': 'improper overshoot',
 'dormindo': 'driver asleep',
 'desobediência à sinalização': 'disobedience to the rules of transit by the driver',
 'defeito mecânico em veículo': 'mecanical problem',
 'não guardar distância de segurança': 'unsafety distance',
 'defeito na via': 'defect on the road',
 'animais na pista': 'animal in the road',        
}

df['accident_cause_simplified'] = df.causa_acidente.apply(lambda x: accident_cause[x.lower().strip()])

In [19]:
clima = {
 'chuva': 'rain',
 'garoa/chuvisco': 'bad weather',
 'céu claro': 'clear sky',
 'nublado': 'cloudy',
 'ignorado': 'notavailable',
 'nevoeiro/neblina': 'bad weather',
 'sol': 'clear sky',
 'vento': 'windy',
 'granizo': 'bad weather',
 'neve': 'bad weather',
 'vento': 'bad weather',
 'ceu claro': 'clear sky',
 'nublado': 'cloudy',
 'chuva': 'rain',
 'nevoeiro/neblina': 'bad weather',
 'sol': 'clear sky',
 'ignorada': 'notavailable',
 'granizo': 'bad weather',
 'indisponivel': 'notavailable',
 'neve': 'bad weather'
}

df['climate'] = df.condicao_metereologica.fillna('indisponivel').apply(lambda x: clima[x.lower().strip()])

In [20]:
df = df[df.climate != 'notavailable']

In [21]:
moment_of_day = {
 'plena noite': 'night',
 'amanhecer': 'dawn',
 'pleno dia': 'day',
 'anoitecer': 'nightfall',
 'plena noite': 'night',
 'pleno dia': 'day',
 'amanhecer': 'dawn',
 'anoitecer': 'nightfall',
 'nan': 'nan'
}
df['moment_of_day'] = df.fase_dia.fillna('nan').apply(lambda x: moment_of_day[x.lower().strip()])

df = df[df.moment_of_day != 'nan']

In [22]:
km_str = df.km.str.replace(',','.')
km_float = km_str.astype(float)

km_float = km_float.fillna(99999)

km_int = km_float.appround(x)))
df['km_int'] = km_int

In [23]:
br_int = df.br.fillna(99999).astype(int)
br_str = br_int.astype(object)

df['br_str'] = br_str

In [24]:
df.columns

Index(['br', 'causa_acidente', 'classificacao_acidente',
       'condicao_metereologica', 'delegacia', 'dia_semana', 'fase_dia',
       'feridos', 'feridos_graves', 'feridos_leves', 'ignorados', 'ilesos',
       'km', 'latitude', 'longitude', 'mortos', 'municipio', 'pessoas',
       'regional', 'sentido_via', 'tipo_acidente', 'tipo_pista', 'tracado_via',
       'uf', 'uop', 'uso_solo', 'veiculos', 'hora', 'ano', 'mes', 'dia',
       'target', 'weekday', 'accident_type', 'accident_type_simplified',
       'accident_cause_simplified', 'climate', 'moment_of_day', 'km_int',
       'br_str'],
      dtype='object')

In [25]:
road_type = {
    'simples': 'simple', 
    'dupla': 'double', 
    'múltipla': 'multiple', 
    'simples': 'simple', 
    'dupla': 'double', 
    'múltipla': 'multiple',
}

df['road_type'] = df.tipo_pista.apply(lambda x: road_type[x.lower().strip()])

In [26]:
road_way = {
    'crescente': 'up', 
    'decrescente': 'down', 
    'não informado': 'notavailable', 
    'crescente': 'up', 
    'decrescente': 'down',
}
df['road_way'] = df.sentido_via.apply(lambda x: road_way[x.strip().lower()])

In [27]:
road_layout = {
    'reta': 'straight',
    'curva': 'curve',
    'não informado': 'notavailable',
    'interseção de vias': 'intersection',
    'rotatória': 'roundabout',
    'desvio temporário': 'temporary detour',
    'viaduto': 'viaduct',
    'ponte': 'bridge',
    'retorno regulamentado': 'return',
    'túnel': 'tunnel',
    'reta': 'straight',
    'curva': 'curve',
    'cruzamento': 'crossing',
}
df['road_layout'] = df.tracado_via.apply(lambda x: road_layout[x.lower().strip()])

In [28]:
df.drop(['causa_acidente', 'dia_semana', 'tipo_acidente', 'classificacao_acidente', 
         'condicao_metereologica', 'fase_dia', 'km', 'br', 'tipo_pista', 'sentido_via',
         'tracado_via'], axis=1, inplace=True)

In [29]:
df.to_csv('../data/datatran_2016-2017.csv', index=False)